# 第5回講義 演習

## 課題. Tensorflowの基礎を学ぶ

In [1]:
import numpy as np
#tensorflowをインポート
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow.examples.tutorials.mnist import input_data

rng = np.random.RandomState(1234)
random_state = 42

/Users/masaki/.pyenv/versions/anaconda3-5.0.0/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 1. Tensorflowの概観

`tf`では, 基本的に以下の流れで機械学習モデルを構築します.

1. プレースホルダーと変数の設定
2. グラフの構築
3. 誤差関数の設定
4. 重みの更新ルールの設定
5. `tf.Session()`を開始して学習
6. 予測

#### 線形回帰の例

In [2]:
tf.reset_default_graph() # グラフのリセット
#これをすれば、メモリ上で構築されているグラフがリセットされるので、新たに始めたいときはこれをする。

# Step1. プレースホルダー・変数の設定
## placeholder: データを流し込む変数. データ毎に変わる
x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')

## Variable: 変数(重み). データ間で共有される
w = tf.Variable(0.0, name='w')
b = tf.Variable(0.0, name='b')

# Step2. グラフの構築
y = w*x + b
#shapeを考慮するといましているのは、ただ単に、1次関数を組んで、単回帰しているだけ。

# Step3. 誤差関数の設定
cost = tf.reduce_mean((y - t)**2)

# Step4. 重みの更新則の設定
gw, gb = tf.gradients(cost, [w, b]) # 勾配の計算
# tf.gradients(関数,　その関数の引数とする変数名(テンソル))
# 今回ならば損失関数を構成するwとbについてのそれぞれの勾配が返される。

updates = [
    w.assign(w - 0.1*gw), # 勾配降下法
    b.assign(b - 0.1*gb)
]
train = tf.group(*updates)

# Step.5. 学習 (y = 2*x + 3)
data_X = np.array([0., 1., 2., 3., 4.])
data_y = np.array([3., 5., 7., 9., 11.])

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 重みの初期化
for i in range(100):
    # trainもtunしているのでwとbが更新されて次のループに入る。
    _cost, _ = sess.run([cost, train], feed_dict={x: data_X, t: data_y})
    if (i+1)%10==0:
        print('iteration:: %d, cost:: %.3f' % (i+1, _cost))

# Step6. 予測
print('pred_y:', sess.run(y, feed_dict={x: [5]}))

sess.close()

iteration:: 10, cost:: 0.464
iteration:: 20, cost:: 0.135
iteration:: 30, cost:: 0.040
iteration:: 40, cost:: 0.012
iteration:: 50, cost:: 0.003
iteration:: 60, cost:: 0.001
iteration:: 70, cost:: 0.000
iteration:: 80, cost:: 0.000
iteration:: 90, cost:: 0.000
iteration:: 100, cost:: 0.000
pred_y: [13.003278]


### 2. プレースホルダー・変数

tfには2種類の変数 (のようなもの) があります. それぞれ以下のように使い分けます.

- `tf.placeholder`: データ間で値が共有されない変数 (入力の`x`, 正解ラベルの `t` などに使用)
- `tf.Variable` : データ間で値が共有される変数 (重みの `W`, `b` など更新されるものに使用）

#### 2.1.  `tf.placeholder`

データを流し込む入り口として使います.
- 変数の型 (`tf.int32`, `tf.float32`) を指定する必要があります
- 実行時にはデータを**`feed_dict`**で渡す必要があります

In [3]:
x = tf.placeholder(tf.float32)

y = x**2
print(y)

with tf.Session() as sess:
    print(sess.run(y, feed_dict={x: 3}))

Tensor("pow_1:0", dtype=float32)
9.0


#### 2.2. `tf.Variable`

値がデータ間で共有されるので, まず初期値を与える必要があります.

- 全ての変数を初期化する場合は`tf.global_variables_initializer()`を使います.
- 個別に変数を初期化する場合は`tf.variables_initializer()`を使い, 引数に初期化したい変数をリストで渡します.

In [4]:
w = tf.Variable(0.0, name='w')

with tf.Session() as sess:
#     print(w.eval())#これはエラーになる#変数を初期化する前に評価しようとしているから
    sess.run(tf.global_variables_initializer())
    print(w.eval()) # print(sess.run(w))でも同じです
    print(sess.run(w))

0.0
0.0


In [5]:
w = tf.Variable(0.0)
b = tf.Variable(1.0)

with tf.Session() as sess:
    sess.run(tf.variables_initializer([w]))
    print(w.eval()) 
#     print(b.eval()) # 初期化していないので, エラーが出ます.
print()
with tf.Session() as sess:
    sess.run(tf.variables_initializer([w, b]))
    print(w.eval())
    print(b.eval())

0.0

0.0
1.0


### 3. 数学

APIは`numpy`と非常に似ています. また, `numpy`と**同じく要素毎に演算**が行われます.

In [6]:
x = tf.placeholder(tf.float32)

exp_x = tf.exp(x)
log_x = tf.log(x)
sqrt_x = tf.sqrt(x)

with tf.Session() as sess:
    print(sess.run([exp_x, log_x, sqrt_x], feed_dict={x: [1,2]}))

[array([2.7182817, 7.389056 ], dtype=float32), array([0.       , 0.6931472], dtype=float32), array([1.       , 1.4142135], dtype=float32)]


- ニューラルネットワーク用の関数は`tf.nn`以下にあります

In [7]:
x = tf.placeholder(tf.float32)

sigmoid_x = tf.nn.sigmoid(x)
tanh_x = tf.nn.tanh(x)
relu_x = tf.nn.relu(x)

with tf.Session() as sess:
    print(sess.run([sigmoid_x, tanh_x, relu_x], feed_dict={x: 1}))

[0.7310586, 0.7615942, 1.0]


- `np.mean, np.sum`等に対応するものは`tf.reduce_mean, tf.reduce_sum`等になります.
- 引数`axis`で指定した軸に沿って演算を行います.

In [8]:
x = tf.placeholder(tf.float32)

sum_x = tf.reduce_sum(x, 0)
mean_x = tf.reduce_mean(x, 0)

with tf.Session() as sess:
    print(sess.run([sum_x, mean_x], feed_dict={x: np.arange(10)}))

[45.0, 4.5]


### 4. 行列・テンソル積

`np` の `dot`, `matmul` に対応するものは `tf.matmul` ですが, 少し挙動が違うので注意する必要があります.

#### 4.1. 行列積

- `tf.matmul` を使用します

In [9]:
a = tf.ones([2,2])
b = tf.ones([2,2])

c = tf.matmul(a, b)

with tf.Session() as sess:
    print(a.eval(),"\n",b.eval(),end="\n\n")
    print(c.eval())

[[1. 1.]
 [1. 1.]] 
 [[1. 1.]
 [1. 1.]]

[[2. 2.]
 [2. 2.]]


- ベクトルに対しても `tf.newaxis` などで**明示的に行列に変換する必要**があります.

In [10]:
a = tf.ones([2,2])
b = tf.ones(2)

# c = tf.matmul(a, b) # エラー# 次元が合わないのでエラーになる
c = tf.matmul(a, b[:, tf.newaxis])

with tf.Session() as sess:
    print(b.eval()  )
    print(c.eval())

[1. 1.]
[[2.]
 [2.]]


#### 5.1. テンソル積

- ```np```と同様に```tf```にも```einsum```があります. 3階以上のテンソルを含む計算はこれを用いるのがわかりやすくベターです

In [11]:
a = tf.ones([2,3,4])
b = tf.ones([2,3])

c = tf.einsum('ijk,ij->k', a, b)
#->の先で何も指定しないときはすべてをまとめて、0次元のテンソルとする。
sum_c = tf.einsum('ijk,ij->', a, b)

with tf.Session() as sess:
    print(c.eval())
    print(sum_c.eval())

[6. 6. 6. 6.]
24.0


### 5. 条件・比較演算子

#### 条件の前にpythonのラムダ式についての復習
```
#lambda式の形式
(lambda 引数:処理内容)(実際に入れる引数)
```

In [12]:
a1=3
a2=4
print((lambda x,y: x*y)(a1, a2))

12


In [13]:
(lambda: a2-a1)()

1

In [14]:
exfunc = lambda: a2-a1

In [15]:
exfunc()

1

要は簡易的な関数宣言

#### tensorflowにおいて

条件演算子は
- `tf.cond(condition, if true(関数), if false(関数))`
です.

In [16]:
x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')

absl = tf.cond(x > y, lambda: x - y, lambda: y - x)

with tf.Session() as sess:
    print(sess.run(absl, feed_dict={x: 100, y:  50}))
    print(sess.run(absl, feed_dict={x:  50, y: 100}))

50.0
50.0


比較演算子は
- `tf.equal`
- `tf.greater` (>でも可)
- `tf.less` (<でも可)

などを使います.

In [17]:
x = tf.placeholder(tf.float32, name='x')
y = tf.placeholder(tf.float32, name='y')

absl = tf.cond(tf.greater(x, y), lambda: x - y, lambda: y - x)

with tf.Session() as sess:
    print(sess.run(absl, feed_dict={x: 100, y:  50}))
    print(sess.run(absl, feed_dict={x:  50, y: 100}))

50.0
50.0


他の言語のfor文に対応するものは`tf.scan`ですが, これはRNNの回で扱います.

### 6. 勾配 (微分) の計算

`tf.gradients`をつかうことで微分を計算することができます.

In [18]:
x = tf.placeholder(tf.float32, name='x')
y = x**2

grads = tf.gradients(y, x)

with tf.Session() as sess:
    print(sess.run(grads, feed_dict={x: 1.}))
    print(sess.run(grads, feed_dict={x: 2.}))

[2.0]
[4.0]


第二引数(`xs`)に複数の変数を指定すると, それぞれに対する偏微分をリストで返します.

In [19]:
x1 = tf.placeholder(tf.float32, name='x1')
x2 = tf.placeholder(tf.float32, name='x2')
y = 3*x1**2 + 2*x2**4

grads = tf.gradients(y, [x1, x2])

with tf.Session() as sess:
    print(sess.run(grads, feed_dict={x1: 1, x2: 2}))
    print(sess.run(grads, feed_dict={x1: 3, x2: 4}))

[6.0, 64.0]
[18.0, 512.0]


In [20]:
x1 = tf.placeholder(tf.float32, name='x1')
x2 = tf.placeholder(tf.float32, name='x2')
y = 3*x1**2 + 2*x2**4

grads = tf.gradients(y, [x1])

with tf.Session() as sess:
    print(sess.run(grads, feed_dict={x1: 1, x2: 2}))
    print(sess.run(grads, feed_dict={x1: 3, x2: 4}))

[6.0]
[18.0]


### 7. 変数 (Variable) の更新

`assign, assign_add, assign_sub` メソッドにより行います.

In [21]:
a = tf.Variable(0.0, name='w')

add_one = a.assign_add(1.) #1を足すというオペレータをつくる

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        print(sess.run(add_one))

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0


複数の更新をまとめる場合は `tf.group` を使用します.

In [22]:
a = tf.Variable(0.0, name='w')
b = tf.Variable(10.0, name='b')

add_one = a.assign_add(1.) 
sub_one = b.assign_sub(1.)#1ずつ引くオペレーターを作っている

updates = [
    add_one,
    sub_one
]

# まとめる
train = tf.group(*updates)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        sess.run(train)
        print('a:', a.eval(), end=',  ')
        print('b:', b.eval())

a: 1.0,  b: 9.0
a: 2.0,  b: 8.0
a: 3.0,  b: 7.0
a: 4.0,  b: 6.0
a: 5.0,  b: 5.0
a: 6.0,  b: 4.0
a: 7.0,  b: 3.0
a: 8.0,  b: 2.0
a: 9.0,  b: 1.0
a: 10.0,  b: 0.0


### 8. TensorBoardによるグラフの表示

TensorFlowで計算グラフの構築方法を扱ってきましたが, ここでは構築した計算グラフの可視化をし, 視覚的に捉えてみましょう.

計算グラフを表示するには, tensorboard.py [\[引用元\]](http://qiita.com/kegamin/items/887c7dfe8bbb76197741) を読み込む必要があります.

tensorboard.pyをimportしたら, `show_graph`関数にグラフを渡すことで可視化できます.

可視化結果はインタラクティブな表示になるので, 拡大や移動, 詳細表示等を試してみましょう.

In [23]:
import tensorboard as tb

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

c = a + b

# with tf.Session() as sess:
#     print(sess.run([c], feed_dict={a:2, b:3}))

# tb.show_graph(sess.graph)    # 単純な足し算のグラフの表示 (がしたいが...)

#ここが動かないので下記で修正

In [24]:
# import tensorboard as tb
#import tensorflow_graph_in_jupyter as tb
from tensorflow_graph_in_jupyter import show_graph
tf.reset_default_graph() # グラフのリセット


a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

c = a + b

with tf.Session() as sess:
    print(sess.run([c], feed_dict={a:2, b:3}))

show_graph(tf.get_default_graph())    # 単純な足し算のグラフの表示 (がしたいが...)

[5.0]


### 9. グラフの管理と整理

#### 9.1. デフォルトグラフ

グラフが表示されたものの, 大変ごちゃごちゃしており, 足し算以外のグラフも表示されてしまったかと思います.

これは, 今回の演習でこれまでに実行された計算グラフがすべて表示されてしまっているためです.

TensorFlowでは何も指定しなければ, デフォルトグラフと呼ばれるグラフ上に計算グラフを構築します.

一度計算グラフ上に配置されたグラフは, そのグラフを使うか使わないかにかかわらず, 全てリセットされることなく蓄積されていきます.

In [25]:
# 今までの実行によりデフォルトグラフ上に溜まったオペレーション
tf.get_default_graph().get_operations()

[<tf.Operation 'Placeholder' type=Placeholder>,
 <tf.Operation 'Placeholder_1' type=Placeholder>,
 <tf.Operation 'add' type=Add>]

In [26]:
tf.global_variables() # Variables

[]

#### 9.2.  `tf.reset_default_graph`関数によるリセット

このままでは, 使わないゴミリソースが蓄積してしまう上, TensorBoardで可視化する際にも無関係のグラフまで表示され見づらくなってしまいます.

特にJupyter等のインタラクティブな環境では全体で1セッションなので, こうした傾向が顕著であり, 途中でリセット処理を書くことが重要です.

対処法としては, 毎回新しくグラフを構築する際に `tf.reset_default_graph()` によりグラフをリセットすることです.

こうすることで毎回クリーンな状態でグラフを構築していくことができます.

In [27]:
tf.reset_default_graph() # グラフのリセット
print(tf.get_default_graph().get_operations())
print(tf.global_variables())

# 再び足し算のグラフを構築・表示
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

c = a + b

with tf.Session() as sess:
    print(sess.run(c, feed_dict={a:2, b:3}))

print(tf.get_default_graph().get_operations())
print(tf.global_variables())
# tb.show_graph(tf.Session().graph)
show_graph(tf.get_default_graph())

[]
[]
5.0
[<tf.Operation 'Placeholder' type=Placeholder>, <tf.Operation 'Placeholder_1' type=Placeholder>, <tf.Operation 'add' type=Add>]
[]


#### 9.3. 複雑なグラフの整理

グラフを表示したとき, 各ノードの名前は基本的に自動で割り振られます.

ただ, これでは少し規模が大きくなるだけですぐにコードとの対応をつけるのが難しくなります.

そこで, 定数・変数やプレースホルダーなどには `name`引数を明示的に指定し, ノード名を与えておきましょう.

In [28]:
# 線形回帰の例

# グラフのリセット
tf.reset_default_graph()

# プレースホルダーと変数の宣言
x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')
W = tf.Variable(tf.random_uniform([5,3], -1.0, 1.0), name='W')
b = tf.Variable(tf.zeros([3]), name='b')

# グラフの構築
y = tf.add(tf.matmul(x, W), b, name='y')

# 誤差関数の定義
loss = tf.reduce_mean((y - t)**2, name='loss')

# tb.show_graph(tf.Session().graph)
show_graph(tf.get_default_graph())

かなりコードと対応がつき, 見やすくなりました.

しかし, 今後より大規模なグラフを扱う際には, 今のままでは頂点が余りにも多くなってしまい, 見ずらくなってしまいます.

そこで頂点をまとめることを考えると, これには tf.name_scope 関数を用います.

まとめられた頂点は, カーソルをかざすと出てくる右上のプラスマークをクリックすることで展開することができます.

In [29]:
# グラフのリセット
tf.reset_default_graph()

x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')

with tf.name_scope('variables'):
    W = tf.Variable(tf.random_uniform([5,3], -1.0, 1.0), name='W')
    b = tf.Variable(tf.zeros([3]), name='b')

with tf.name_scope('model'):
    y = tf.add(tf.matmul(x, W), b, name='y')

with tf.name_scope('training'):
    loss = tf.reduce_mean(tf.square(y - t), name='loss')

# tb.show_graph(tf.Session().graph)
show_graph(tf.get_default_graph())

#### 9.4. グラフの切り分け

デフォルトグラフではなく, 明示的にグラフオブジェクトを作成し, その上にグラフを構築していくことでグラフ環境を他と分けることもできます.

これは複数のグラフを構築していきたいときなどに便利です.

In [30]:
tf.reset_default_graph() # グラフのリセット

g0 = tf.get_default_graph() # デフォルトグラフオブジェクトを取得することも可能

##----------------------------------------------------##
g1 = tf.Graph() # グラフオブジェクトの作成1

a = tf.constant(2, name='a0') # これはdefault graphへの配置になるので注意
b = a**a

# with文で、指定したグラフオブジェクトにグラフを構築することが可能
with g1.as_default(): # デフォルトに設定した上で, グラフを構築・操作
    a = tf.constant(2, name='a')
    b = a**a

g2 = tf.Graph() # グラフオブジェクトの作成2

with g2.as_default(): # デフォルトに設定し, グラフを構築
    a = tf.constant(4, name='a')
    x = tf.constant(3, name='x')
    y = a**x

In [31]:
# tb.show_graph(g0)
show_graph(g0)

In [32]:
with tf.Session(graph=g1) as sess:
    print(sess.run(b))
# tb.show_graph(g1)

show_graph(g1)

4


In [33]:
with tf.Session(graph=g2) as sess:
    print(sess.run(y))
# tb.show_graph(g2)
show_graph(g2)

64


※注意

ここで紹介している方法は, グラフの表示のみに対応しています.

TensorBoardそのものはグラフ以外にも, 学習中のパラメータの変化など様々な可視化に対応しているのですが,

Jupyter上では対応が進んでおらず, TensorBoardをフルに活用できないのです.

Jupyterを用いずに手元の環境で行う場合, およそ次の方法でフルのTensorBoardを使用できます. (tensorboard.pyは不要です)

1. tf.Session 中で tf.summary.FileWriter 関数によりログの出力を設定

2. ターミナルで "tensorboard --logdir= (ログの出力先) " を実行

3. localhost:6006にブラウザからアクセス

より興味が湧いた方はぜひ手元の環境で下の公式情報を参考に他の可視化についても試してください.

参考:
- https://www.tensorflow.org/get_started/summaries_and_tensorboard
- https://www.tensorflow.org/get_started/embedding_viz
- https://www.tensorflow.org/get_started/graph_viz

### 10. 自動微分を使ったロジスティック回帰の実装

データセットに OR を用いてロジスティック回帰を実装してみましょう. パラメータの勾配の計算には `tf` の自動微分機能 `tf.gradients` を使ってみましょう

数式

- 予測確率の計算
$$
    y = \sigma({\bf W}^{\mathrm{T}}{\bf x} + {\bf b})
$$
- 誤差関数: 交差エントロピー
$$
    E = -\sum^N_{i=1} [t_i \log y_i + (1 - t_i) \log (1 - y_i) ]
$$
- 勾配降下法によるパラメータの更新 ($\epsilon$: 学習率)
$$
\begin{align*}
    {\bf W}^{(l)} &\leftarrow {\bf W}^{(l)} - \epsilon \frac{\partial E}{\partial {\bf W}^{(l)}}  \\
    {\bf b}^{(l)} &\leftarrow {\bf b}^{(l)} - \epsilon \frac{\partial E}{\partial {\bf b}^{(l)}}
\end{align*}
$$

In [34]:
tf.reset_default_graph() # グラフのリセット

# Step1. プレースホルダーと変数の定義
## プレースホルダー
x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')

## 変数
# rngは乱数のseed
with tf.name_scope('variables'):
    W = tf.Variable(rng.uniform(low=-0.08, high=0.08, size=(2, 1)).astype('float32'), name='W')
    b = tf.Variable(np.zeros(1).astype('float32'), name='b')

# Step2. グラフの構築
y = tf.nn.sigmoid(tf.matmul(x, W) + b, name="y")

# Step3. 誤差関数の定義
# cost = -tf.reduce_mean(t*tf.log(y) + (1 - t)*tf.log(1 - y))
# 大きくなりすぎたり小さくなりすぎないように勾配をクリッピングしている
with tf.name_scope("cost"):
    cost = -tf.reduce_mean(t*tf.log(tf.clip_by_value(y, 1e-10, 1.0)) + (1 - t)*tf.log(tf.clip_by_value(1 - y, 1e-10, 1.0))) # tf.log(0)によるnanを防ぐ

# Step4. 更新則の設定
with tf.name_scope("update"):
    gW, gb = tf.gradients(cost, [W, b])
    updates = [
        W.assign_add(-0.01*gW),
        b.assign_add(-0.01*gb)
    ]
    train = tf.group(*updates)

# OR
train_X = np.array([[0, 1], [1, 0], [0, 0], [1, 1]])
train_y = np.array([[1], [1], [0], [1]])

# Step5. 学習
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        _cost, _ = sess.run([cost, train], feed_dict={x: train_X, t: train_y})
        if (i+1)%1000==0:
            print(_cost)

show_graph(tf.get_default_graph())

0.34381974
0.26790747
0.21812478
0.18308824
0.15722485
0.1374165
0.12180197
0.10920583
0.09884994
0.09019965


### 11. 自動微分を使った多層パーセプトロン (Multilayer perceptron, MLP) の実装

データセットに MNIST を用いて MLP を実装してみましょう. パラメータの勾配の計算には `tf` の自動微分機能 `tf.gradients` を使ってみましょう

数式

- 順伝播
$$
\begin{align*}
    {\bf u}^{(1)} &= {\bf W}^{(1)\mathrm{T}} {\bf x} + {\bf b}^{(1)} \\
    {\bf z}^{(1)} &= \sigma({\bf u}^{(1)}) \\
    {\bf u}^{(2)} &= {\bf W}^{(2)\mathrm{T}} {\bf z^{(1)}} + {\bf b}^{(2)} \\
    {\bf y} &= \mathrm{softmax} ({\bf u}^{(2)})
\end{align*}
$$
- 誤差関数: 多クラス交差エントロピー
$$
    E = -\sum^N_{i=1} \sum^K_{k=1} t_{ik} \log y_{ik}
$$
- 勾配降下法によるパラメータの更新 ($\epsilon$: 学習率)
$$
\begin{align*}
    {\bf W}^{(l)} &\leftarrow {\bf W}^{(l)} - \epsilon \frac{\partial E}{\partial {\bf W}^{(l)}}  \\
    {\bf b}^{(l)} &\leftarrow {\bf b}^{(l)} - \epsilon \frac{\partial E}{\partial {\bf b}^{(l)}}
\end{align*}
$$

In [35]:
tf.reset_default_graph() # グラフのリセット

# Step1. プレースホルダーと変数の定義
## Placeholders
x = tf.placeholder(tf.float32, [None, 784])
t = tf.placeholder(tf.float32, [None, 10])

## 変数
W1 = tf.Variable(rng.uniform(low=-0.08, high=0.08, size=(784, 200)).astype('float32'), name='W1')
b1 = tf.Variable(np.zeros(200).astype('float32'), name='b1')
W2 = tf.Variable(rng.uniform(low=-0.08, high=0.08, size=(200, 10)).astype('float32'), name='W2')
b2 = tf.Variable(np.zeros(10).astype('float32'), name='b2')
params = [W1, b1, W2, b2]

# Step2. グラフの定義
u1 = tf.matmul(x, W1) + b1
z1 = tf.nn.sigmoid(u1)
u2 = tf.matmul(z1, W2) + b2
y = tf.nn.softmax(u2)

# Step3. 誤差関数の定義
# cost = -tf.reduce_mean(tf.reduce_sum(t*tf.log(y)))
cost = -tf.reduce_mean(tf.reduce_sum(t*tf.log(tf.clip_by_value(y, 1e-10, 1.0)))) # tf.log(0)によるnanを防ぐ

# Step4. 更新則の設定
gW1, gb1, gW2, gb2 = tf.gradients(cost, params)
updates = [
    W1.assign_add(-0.01*gW1),
    b1.assign_add(-0.01*gb1),
    W2.assign_add(-0.01*gW2),
    b2.assign_add(-0.01*gb2)
]

train = tf.group(*updates)

valid = tf.argmax(y, 1)

# MNIST
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_X, mnist_y = mnist.train.images, mnist.train.labels
train_X, valid_X, train_y, valid_y = train_test_split(mnist_X, mnist_y, test_size=0.1, random_state=random_state)

n_epochs = 10
batch_size = 100
n_batches = train_X.shape[0] // batch_size

# Step5. 学習
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            sess.run(train, feed_dict={x: train_X[start:end], t: train_y[start:end]})
        pred_y, valid_cost = sess.run([valid, cost], feed_dict={x: valid_X, t: valid_y})
        print('EPOCH:: %i, Validation cost: %.3f, Validation F1: %.3f' % (epoch + 1, valid_cost, f1_score(np.argmax(valid_y, 1).astype('int32'), pred_y, average='macro')))

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
EPOCH:: 1, Validation cost: 1482.452, Validation F1: 0.916
EPOCH:: 2, Validation cost: 1174.606, Validation F1: 0.937
EPOCH:: 3, Validation cost: 928.269, Validation F1: 0.949
EPOCH:: 4, Validation cost: 789.311, Validation F1: 0.958
EPOCH:: 5, Validation cost: 730.649, Validation F1: 0.959
EPOCH:: 6

### 12. 参考資料

1. [Tensorflow Documentation](https://www.tensorflow.org/api_docs/)
2. [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/mandelbrot)
2. [CS 20SI: Tensorflow for Deep Leaning Research](http://web.stanford.edu/class/cs20si/)
3. [CS224d: Tensorflow Tutorial](https://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf)

いや、生tensorflow記述量多すぎ！！！！